In [1]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import requests
import time
import json

In [2]:
client = MongoClient("mongodb://localhost:27017/")
db = client.test

In [3]:
def temp_store_memory(agent_id, agents_involved, content):
    memory = {
        "agent_id": agent_id, #!!!
        "agents_involved": agents_involved, #!!!
        "timestamp": datetime.now(), #!!!
        "content": content, #!!!
    }
    db.agent_memories.insert_one(memory)

In [4]:
temp_store_memory("agent_001",
                  ["agent_002"],
                  "I am cooking pasta.")

In [5]:
def temp_retrieve_memories(agent_id):
    memories = db.agent_memories.find({
        "agent_id": agent_id
    }).sort("timestamp", -1)
    return list(memories)

def temp_retrieve_memories_concerning(agent_id):
    memories = db.agent_memories.find({
        "$or": [
            {"agent_id": agent_id},
            {"agents_involved": agent_id}
        ]
    }).sort("timestamp", -1)
    return list(memories)

print(temp_retrieve_memories("agent_001"))

[{'_id': ObjectId('67530145a424751c190109b3'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 6, 14, 51, 1, 785000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('6752e8fe2e49c167149da24a'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 6, 13, 7, 26, 67000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('6752e7f11600ddc2480214c3'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 6, 13, 2, 57, 143000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('6731eab8ebb492eb6820de74'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 11, 11, 12, 30, 0, 349000), 'content': 'Had this conversation with Eliana: [\'Sami: Sami\\\'s personality is quite exhausting, don\\\'t you think? He always starts every sentence with "I\\\'m tired of..." which makes me roll my eyes.\', \'0: Elia

In [6]:
class Agent:
    def __init__(self, name, agent_id, user_input, gender):
        self.name = name
        self.agent_id = agent_id 
        self.user_input = user_input
        self.gender = gender
        self.opinions = {}
        
    def __str__(self):
        return f"Agent {self.name} (ID: {self.agent_id}) (Description: {self.user_input}) (Opinions: {self.opinions})"
    
    def name(self):
        return self.name
    
    def agent_id(self):
        return self.agent_id
    
    def user_input(self):
        return self.user_input
    
    def gender(self):
        return self.gender

In [ ]:
Sami = Agent("Sami", "agent_001", "Sami is a man who likes horses and love to go hiking. He also loves fishing. He has an opinion on everything and is kind most of the time.", "male")

Eliana = Agent("Eliana", "agent_002", "Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about ducks. She is the sweetest person on earth and would never lie or attack someone.", "female")

Alice = Agent("Alice", "agent_003", "Alice is a woman who loves cats and crochet. She doesn't like cooking at all. She is nice to people unless they are unkind to her first.", "female")

Sami.opinions["Eliana"] = "He likes Eliana and thinks her cooking advices are great"
Sami.opinions["Alice"] = "He enjoys having conversations with her about his hobbies"

Eliana.opinions["Sami"] = "She thinks Sami is cool and she likes his cooking"
Eliana.opinions["Alice"] = "She's happy to have found someone who likes animals as much as she does"

Alice.opinions["Sami"] = "She likes hearing him talk about his hiking adventures"
Alice.opinions["Eliana"] = "She likes that she's a kind and calm person"

print(Sami)
print(Eliana)
print(Alice)

Agent Sami (ID: agent_001) (Description: Sami is a man who likes horses and love to go hiking. He also loves fishing. He has an opinion on everything and is kind most of the time.) (Opinions: {'Eliana': 'He likes Eliana and thinks her cooking advices are great', 'Alice': 'He enjoys having conversations with her about his hobbies'})
Agent Eliana (ID: agent_002) (Description: Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about ducks. She is the sweetest person on earth and would never lie or attack someone.) (Opinions: {'Sami': 'She thinks Sami is cool and she likes his cooking', 'Alice': "She's happy to have found someone who likes animals as much as she does"})
Agent Alice (ID: agent_003) (Description: Alice is a woman who loves cats and crochet. She doesn't like cooking at all. She is nice to people unless they are unkind to her first.) (Opinions: {'Sami': 'She likes hearing him talk about his hiking adventures', 'Eliana': "She likes that she's a kind an

In [47]:
def make_initial_prompt(agent1, agent2, subject):
    message_content = f"""
            Context:
            Here is a description of {agent1.name}: {agent1.user_input}.
            Here is a description of {agent2.name}: {agent2.user_input}.
            Here is what {agent1.name} thinks about {agent2.name}:
            {agent1.opinions[agent2.name]}
            Here is what {agent2.name} thinks about {agent1.name}:
            {agent2.opinions[agent1.name]}
            The context of this conversation is: {subject}
            Start directly by a quick sentence describing the scene, what the agents were doing before starting the conversation and their main emotion, without an introduction.
            Then, create the conversation they had:

            {agent1.name}: "
        """
    return message_content

def make_subject(memory):
    message_content = f"""
            Context:
            This is the memory of a conversation between 2 people:
            {memory}
            Make a summary in 1 sentence or 2 with the important points and details of the conversation (for example, if they are talking about doing an activity or a dish together, keep it in memory), ignorig the first sentence.
            Start directly with the summary, not with a phrase like "here is a summary".
        """
    return message_content

def extract_emotion(memory, agent_concerned, other):
    message_content = f"""
    Agent1: {agent_concerned.name}, {agent_concerned.gender}, {agent_concerned.user_input}
    Agent2: {other.name}, {other.gender}, {other.user_input}
    Conversation: "{memory}"
    Possible emotions: ["happy", "sad", "angry", "neutral", "fearful"]

    Analyze the information about the agents and their conversation, and identify the primary emotion felt by Agent1 as a result of this interaction.
    The response MUST be a SINGLE word from the provided list of emotions, only from the provided list, nothing more.
    """
    return message_content

def update_opinion(memory, agent_concerned, other):
    message_content = f"""
    {agent_concerned.name}: {agent_concerned.gender}, {agent_concerned.user_input}
    {other.name}: {other.gender}, {other.user_input}
    Conversation: "{memory}"

    Here is the conversation that happened between {agent_concerned.name} and {other.name}.
    Summarize what {agent_concerned.name} thought about {other.name} in one short sentence. The sentence needs to be in third person:
    """
    return message_content

def update_opinion2(opinion, agent_concerned, other, emotion=None):
    message_content = f"""
    {agent_concerned.name}: {agent_concerned.gender}, {agent_concerned.user_input}
    {other.name}: {other.gender}, {other.user_input}
    Last opinion of {agent_concerned.name} about {other.name}: {agent_concerned.opinions[other.name]}
    Analyzed phrase or conversation: "{opinion}"
    {f'Emotion felt by {agent_concerned.name}: "{emotion}"' if emotion else ''}

    Based on the information about both agents and the provided text, update the opinion of {agent_concerned.name} about {other.name}.
    If an emotion is specified, take it into account to adjust the tone or content of this opinion.
    If a part of the old opinion is correct, keep it in the new opinion.
    Juste give the new opinion, nothing else.
    """
    return message_content

def extract_location(memory):
    message_content = f"""
    Conversation: "{memory}"
    
    Based on the context of the conversation (the first sentence), extract the location of the interaction, in a single word.
    """
    return message_content

In [76]:
url = "http://localhost:1234/v1/chat/completions"
headers = {
    "Content-Type": "application/json"
}

def chat(agent1, agent2, subject, use_memory = True, use_location = None):
    # First agent's turn
    memories = temp_retrieve_memories(agent1.agent_id)
    if (len(memories) > 0 and use_memory):
        subject_prompt = make_subject(memories[0])
        data = {
            "model": "llama-3.2-3b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": subject_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }
        response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
        subject = "Here is the resume of the past conversation: " + response + " They meet some time later"+ {'' if use_location==None else "at "+use_location}
        print("Subject :", subject)
    
    message_prompt = make_initial_prompt(agent1, agent2, subject)

    data = {
            "model": "llama-3.2-3b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": message_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }

    response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
    print(response)
    #temp_store_memory(agent1.agent_id, [agent2.agent_id], str(response))
    #temp_store_memory(agent2.agent_id, [agent1.agent_id], str(response))
    return response

In [78]:
dialog = chat(Eliana, Alice, "They are talking about life", False, "park")

They sat on a bench in a quiet park, watching as children played tag near the pond.

Eliana's eyes shone with warmth. "I'm so glad I found someone who shares my passion for animals."

Alice smiled softly, her brow furrowed in concern. "You're really happy to have met me?"

Eliana nodded enthusiastically. "Yes! You bring joy into my life, and that's what matters most. Someone as kind as you is a breath of fresh air."


In [ ]:
location_prompt = extract_location(dialog)
data = {
    "model": "llama-3.2-3b-instruct",
    "messages": [
        {"role": "system", "content": ""},
        {"role": "user", "content": location_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
    }
location = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
print("location:", location)

location: PARK


In [53]:
subject_prompt = extract_emotion(dialog, Alice, Eliana)
data = {
    "model": "llama-3.2-3b-instruct",
    "messages": [
        {"role": "system", "content": ""},
        {"role": "user", "content": subject_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
    }
emotion = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
print("emotion:", emotion)

subject_prompt = update_opinion(dialog, agent_concerned=Alice, other=Eliana)
data = {
    "model": "llama-3.2-3b-instruct",
    "messages": [
        {"role": "system", "content": ""},
        {"role": "user", "content": subject_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
    }
opinion = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
print("\nOpinion about the conversation:", opinion)

subject_prompt = update_opinion2(opinion, agent_concerned=Alice, other=Eliana, emotion=emotion)
data = {
    "model": "llama-3.2-3b-instruct",
    "messages": [
        {"role": "system", "content": ""},
        {"role": "user", "content": subject_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
    }
response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
print("\nnew opinion:", response)

emotion: angry

Opinion about the conversation: Here is a summary of what Alice thought about Eliana in one short sentence:

Alice thought that Eliana was being overly kind and naive, dismissing her frustration and refusal to consider her duck-related suggestions as simply not being taken seriously enough.

(Note: This summary assumes that Alice's characterization would be based on the information provided in the story. If there is more context or further development of Alice's character, it may be possible to provide a more nuanced summary.)

new opinion: Eliana was being overly kind and naive.

(I kept her kindness and calm nature intact while updating her characterization to reflect Alice's emotion)
